In [ ]:
import pandas as pd
# Load CustomerDemographic dataset
customer_demographic_df = pd.read_excel('KPMG_VI.xlsx', sheet_name='CustomerDemographic')

# Load Transactions dataset
transactions_df = pd.read_excel('KPMG_VI.xlsx', sheet_name='Transactions')

# Load CustomerAddress dataset
customer_address_df = pd.read_excel('KPMG_VI.xlsx', sheet_name='CustomerAddress')

testset = pd.read_excel('KPMG_VI.xlsx', sheet_name='TestSet')

# Merge CustomerDemographic with Transactions on 'customer_id'
merged_df = pd.merge(customer_demographic_df, transactions_df, on='customer_id', how='inner')

# Merge with CustomerAddress on 'customer_id'
merged_df = pd.merge(merged_df, customer_address_df, on='customer_id', how='inner')

print(merged_df.head())  # Display the first few rows
print(merged_df.info())  # Display summary information


In [ ]:
merged_df.to_csv('train_dataset_KPMG.csv', index=False)

In [ ]:
# Load the merged dataset
df = pd.read_csv('train_dataset_KPMG.csv')

# Change data type of 'DOB' and 'transaction_date' columns to datetime
df['DOB'] = pd.to_datetime(df['DOB'])
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Verify the updated data types
print(df.dtypes)


In [ ]:
def separate_columns(data):
    numerical_columns = []
    categorical_columns = []
    datetime_columns = []

    for col in data.columns:
        if pd.api.types.is_numeric_dtype(data[col]):
            numerical_columns.append(col)
        elif pd.api.types.is_datetime64_any_dtype(data[col]):
            datetime_columns.append(col)
        else:
            categorical_columns.append(col)

    return numerical_columns, categorical_columns, datetime_columns

# Separate columns
numerical_cols, categorical_cols, datetime_cols = separate_columns(df)

print("Numerical Columns:")
print(numerical_cols)

print("\nCategorical Columns:")
print(categorical_cols)

print("\nDatetime Columns:")
print(datetime_cols)


In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize distribution using histograms
df[numerical_cols].hist(bins=20, figsize=(10, 6))
plt.tight_layout()
plt.show()

# Visualize distribution using box plots
plt.figure(figsize=(10, 6))
sns.boxplot(data=df[numerical_cols])
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display summary statistics
summary_stats = df[numerical_cols].describe()
print(summary_stats)


In [ ]:
# Visualize distribution using bar plots
for col in categorical_cols:
    plt.figure(figsize=(8, 6))
    sns.countplot(data=df, x=col)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Display value counts for each category
for col in categorical_cols:
    value_counts = df[col].value_counts()
    print(f"Value counts for {col}:\n{value_counts}\n")


In [ ]:
from datetime import datetime

# Calculate age based on DOB and current date
current_date = datetime.now()
df['DOB'] = pd.to_datetime(df['DOB'])  # Ensure DOB column is datetime type
df['age'] = (current_date - df['DOB']).dt.days // 365

# Display the updated DataFrame
print(df[['customer_id', 'DOB', 'age']].head())


In [ ]:
# Calculate frequency of purchases for each customer
purchase_frequency = df.groupby('customer_id')['transaction_id'].count()
df = df.merge(purchase_frequency, how='left', on='customer_id')
df.rename(columns={'transaction_id_x': 'transaction_id', 'transaction_id_y': 'purchase_frequency'}, inplace=True)

# Calculate average transaction amount for each customer
average_transaction_amount = df.groupby('customer_id')['list_price'].mean()
df = df.merge(average_transaction_amount, how='left', on='customer_id')
df.rename(columns={'list_price_x': 'list_price', 'list_price_y': 'avg_transaction_amount'}, inplace=True)

# Display the updated DataFrame
print(df[['customer_id', 'purchase_frequency', 'avg_transaction_amount']].head())


In [ ]:
from datetime import datetime


# Define age brackets
age_bins = [0, 18, 30, 45, 60, 100]
age_labels = ['<18', '18-30', '31-45', '46-60', '60+']
df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels, right=False)

# Display the updated DataFrame
print(df[['customer_id', 'DOB', 'age', 'age_group']].head())


In [ ]:
df.to_csv('train_dataset_KPMG_after_feature_engg.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.hist(df['past_3_years_bike_related_purchases'], bins=20, color='blue', alpha=0.7)
plt.title('Distribution of Past 3 Years Bike Related Purchases')
plt.xlabel('Past 3 Years Bike Related Purchases')
plt.ylabel('Frequency')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot([df['list_price'], df['standard_cost']], labels=['List Price', 'Standard Cost'])
plt.title('Distribution of List Price and Standard Cost')
plt.ylabel('Amount')
plt.show()


In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x='wealth_segment', y='tenure', data=df)
plt.title('Average Tenure Across Wealth Segments')
plt.xlabel('Wealth Segment')
plt.ylabel('Average Tenure')
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(df['age'], bins=20, color='green', alpha=0.7)
plt.title('Distribution of Customer Ages')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='gender', y='purchase_frequency', hue='wealth_segment', data=df)
plt.title('Purchase Frequency by Gender and Wealth Segment')
plt.xlabel('Gender')
plt.ylabel('Purchase Frequency')
plt.legend(title='Wealth Segment')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='job_industry_category', y='avg_transaction_amount', data=df)
plt.title('Average Transaction Amount by Job Industry Category')
plt.xlabel('Job Industry Category')
plt.ylabel('Average Transaction Amount')
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x='gender', data=df)
plt.title('Distribution of Gender')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(y='job_title', data=df, order=df['job_title'].value_counts().index[:10])
plt.title('Top 10 Most Common Job Titles')
plt.xlabel('Count')
plt.ylabel('Job Title')
plt.show()

plt.figure(figsize=(10, 6))
sns.countplot(y='job_industry_category', data=df, order=df['job_industry_category'].value_counts().index)
plt.title('Distribution of Job Industry Categories')
plt.xlabel('Count')
plt.ylabel('Job Industry Category')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x='job_industry_category', hue='wealth_segment', data=df)
plt.title('Wealth Segment Distribution Across Job Industry Categories')
plt.xlabel('Job Industry Category')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.legend(title='Wealth Segment')
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='owns_car', y='purchase_frequency', data=df)
plt.title('Purchase Frequency by Car Ownership')
plt.xlabel('Owns Car')
plt.ylabel('Purchase Frequency')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='deceased_indicator', data=df, hue='online_order')
plt.title('Distribution of Deceased Indicator and Online Order')
plt.xlabel('Deceased Indicator')
plt.ylabel('Count')
plt.legend(title='Online Order')
plt.show()

plt.figure(figsize=(8, 6))
sns.boxplot(x='deceased_indicator', y='purchase_frequency', data=df)
plt.title('Purchase Frequency by Deceased Indicator')
plt.xlabel('Deceased Indicator')
plt.ylabel('Purchase Frequency')
plt.show()


In [ ]:
correlation_matrix = df[['past_3_years_bike_related_purchases', 'tenure', 'age']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()


In [ ]:
sns.scatterplot(x='tenure', y='past_3_years_bike_related_purchases', data=df)
plt.title('Relationship between Tenure and Purchase Activity')
plt.xlabel('Tenure')
plt.ylabel('Past 3 Years Bike Purchases')
plt.show()


In [ ]:
correlation = df['list_price'].corr(df['standard_cost'])
sns.scatterplot(x='list_price', y='standard_cost', data=df)
plt.title('Correlation between List Price and Standard Cost')
plt.xlabel('List Price')
plt.ylabel('Standard Cost')
plt.show()


In [ ]:
correlation_matrix = df[['age', 'purchase_frequency', 'avg_transaction_amount']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

sns.scatterplot(x='age', y='purchase_frequency', data=df)
plt.title('Relationship between Age and Purchase Frequency')
plt.xlabel('Age')
plt.ylabel('Purchase Frequency')
plt.show()

sns.scatterplot(x='age', y='avg_transaction_amount', data=df)
plt.title('Relationship between Age and Average Transaction Amount')
plt.xlabel('Age')
plt.ylabel('Average Transaction Amount')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='age_group', y='purchase_frequency', hue='gender', data=df)
plt.title('Purchase Frequency by Age Group and Gender')
plt.xlabel('Age Group')
plt.ylabel('Purchase Frequency')
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x='age_group', y='avg_transaction_amount', hue='wealth_segment', data=df)
plt.title('Average Transaction Amount by Age Group and Wealth Segment')
plt.xlabel('Age Group')
plt.ylabel('Average Transaction Amount')
plt.show()


In [ ]:
# Drop date columns
date_columns = ['DOB', 'transaction_date']
df = df.drop(date_columns, axis=1)


In [ ]:
# Perform one-hot encoding on nominal categorical columns
nominal_categorical_columns = ['gender', 'job_title', 'job_industry_category', 'wealth_segment', 'owns_car', 'state']
df = pd.get_dummies(df, columns=nominal_categorical_columns, drop_first=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for ordinal categorical columns
ordinal_categorical_columns = ['deceased_indicator', 'product_line', 'product_class', 'product_size']
label_encoder = LabelEncoder()

# Apply label encoding to each column
for column in ordinal_categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])


In [ ]:
df.info()


In [ ]:
# Assuming your DataFrame is named 'df'
numerical_columns = df.select_dtypes(include=['int64', 'float64'])  # Select numerical columns

X_numerical = numerical_columns.drop(['avg_transaction_amount'], axis=1)  # Drop the target column if it's included

# Separate the target variable
y = numerical_columns['avg_transaction_amount']

# Now X_numerical contains only the numerical features, and y contains the target variable


In [ ]:
from sklearn.model_selection import train_test_split

# Split further into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_numerical, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
X_train

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import mean_squared_error, silhouette_score

def choose_best_algorithm(X_train, X_val, y_train, y_val, clustering=False):
    if not clustering:
        # Initialize models
        models = {
            'Linear Regression': LinearRegression(),
            'Random Forest Regression': RandomForestRegressor(),
            'XGBoost Regression': XGBRegressor()
        }

        # Train and evaluate models
        results = {}
        for name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            rmse = mean_squared_error(y_val, y_pred, squared=False)
            results[name] = rmse

        # Choose the best performing algorithm
        best_algorithm = min(results, key=results.get)
        return best_algorithm, results

    else:
        # Initialize clustering models
        models = {
            'K-Means': KMeans(),
            'Hierarchical Clustering': AgglomerativeClustering()
        }

        # Evaluate clustering models using silhouette score
        results = {}
        for name, model in models.items():
            labels = model.fit_predict(X_train)
            score = silhouette_score(X_train, labels)
            results[name] = score

        # Choose the best performing clustering algorithm
        best_algorithm = max(results, key=results.get)
        return best_algorithm, results

# Call the function for regression algorithm selection
best_regression_algorithm, regression_results = choose_best_algorithm(X_train, X_val, y_train, y_val)

# Call the function for clustering algorithm selection
best_clustering_algorithm, clustering_results = choose_best_algorithm(X_train, X_val, y_train, y_val, clustering=True)

print("Best Regression Algorithm:", best_regression_algorithm)
print("Regression Algorithm Results:", regression_results)
print("Best Clustering Algorithm:", best_clustering_algorithm)
print("Clustering Algorithm Results:", clustering_results)
